In [1]:
import pandas as pd
import numpy as np

Os dados que vamos importar representam todas as mátriculas de alunos de Computação em alguma disciplina desde 2003.
Cada observação dos dados tem as seguintes características:
- PERIODO
- COD_DISCIPLINA
- NOME_DISCIPLINE
- MAT_ALUNO.

Ao todo foram analisados (as) mais de 1700 alunos (as).

In [2]:
matriculas_cc = pd.read_csv('dados/matriculas_cc.csv', sep=',')

Calculando quantas vezes cada par de alunos (as) estudou juntos

In [3]:
PERIODO, CODIGO, DISCIPLINA, MATRICULA = 0, 1, 2, 3

network = {}
for group, obs in matriculas_cc.groupby(['PERIODO', 'CODIGO']):
    
    for value_x in obs.get_values():
        mat_x = value_x[MATRICULA]
        
        for value_y in obs.get_values():
            mat_y = value_y[MATRICULA]
            
            if mat_x != mat_y:
                
                if mat_x not in network:
                    network[mat_x] = {}
                
                if mat_y not in network[mat_x]:
                    network[mat_x][mat_y] = 0
                
                network[mat_x][mat_y] += 1

Para cada matrícula, filtram-se os (as) alunos (as) com quem mais estudou-se nas mesmas turmas.

In [4]:
same_num_connections = {}

for mat, connections in network.iteritems():
    
    my_connections_avg = np.mean(connections.values())
    my_connections_max = max(connections.values())
    my_n_connections = (my_connections_avg + my_connections_max) / 2
        
    my_same_n_connections = set(filter(lambda x: connections[x] >= my_n_connections, connections))
    same_num_connections[mat] = my_same_n_connections

As reais turmas de alunos (as) que sempre estudaram juntos e calculado usando o algoritmo Depth-First Search para grafos. Rodamos o DFS para encontrar componentes, em que cada componente corresponde justamente a um grupo de alunos (as) que se matricularam muitas vezes juntos.

In [5]:
def is_on_group(set_mother, me):
    
    if set_mother not in same_num_connections[me]:
        return False
    
    n_containing = len(list(filter(lambda adj: me in same_num_connections[adj], same_num_connections[set_mother])))
    return float(n_containing) / len(same_num_connections[set_mother]) >= 0.9    
    
def dfs(m):
    visited[m] = True
    my_components.append(m)
    for adj in same_num_connections[m]:
        if not visited[adj] and is_on_group(m, adj):
            dfs(adj)
            
components = []
visited = {m:False for m in network}
for mat, values in same_num_connections.iteritems():
    if not visited[mat]:
        my_components = []
        dfs(mat)
        components.append(my_components)

#### Constructing .csv to simple network

Nós também salvaremos os (as) alunos (as) que formaram algum grupo de tamanho 2 ou maior. Precisaremos saber dessa informação mais à frente, quando formos construir os dados para networks que possam ser melhor customizadas.

In [6]:
clusters = []
has_cluster = []

for component in components:
    
    if len(component) > 1:
        for i in range(len(component)-1, -1, -1):
            
            new_row = {'source': component[i],
                       'target': component[i-1]}
            clusters.append(new_row)
            
            has_cluster.append(component[i])
            has_cluster.append(component[i-1])

has_cluster = set(has_cluster)

In [7]:
output = pd.DataFrame(columns=('source', 'target'))
output = pd.DataFrame(clusters)
output.to_csv('dados/samplenetwork.csv', index=False)

#### Constructing datas (.csv's) to more customizing networks.

In [8]:
alunos_cc = pd.read_csv('dados/alunos_cc.csv', sep=',')

In [9]:
i_group = 0
i_row_index = 0
nodes = []
row_index = {}
MATRICULA = 1

for group, obs in alunos_cc.groupby(['PERIODO_INGRESSO']):
    
    i_group += 1
    
    for row_value in obs.get_values():
        
        matricula = row_value[MATRICULA]
        
        if matricula in has_cluster:
            row_index[matricula] = i_row_index
            new_row = {'name': group, 'group': i_group, 'size': 1}
            nodes.append(new_row)
            i_row_index += 1

In [10]:
links = []

for component in components:
    if len(component) > 1:
        for i in range(len(component)-1, -1, -1):
            mat_x = component[i]
            mat_y = component[i-1]
            node_x = row_index[mat_x]
            node_y = row_index[mat_y]
            new_row = {'source': node_x, 'target': node_y, 'value': 1}
            links.append(new_row)

In [11]:
output = pd.DataFrame(columns=('name', 'group', 'value'))
output = pd.DataFrame(nodes)
output.to_csv('dados/network_nodes.csv', index=False)

output = pd.DataFrame(columns=('source', 'target', 'value'))
output = pd.DataFrame(links)
output.to_csv('dados/network_links.csv', index=False)